In [1]:
import json
import pandas as pd

In [2]:
with open('data/fantasypros/players/players.json', 'r') as f:
    players_json = json.load(f)
    
players = pd.DataFrame(players_json['players'])

In [3]:
reddit_comments = pd.read_json('data/reddit/reddit_comments.json')
reddit_submissions = pd.read_json('data/reddit/reddit_submissions.json')
tweets = pd.read_csv('data/twitter/tweets.csv')

In [4]:
# projections and consensus
positions = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']

projections_dfs = []
consensus_dfs = []
for position in positions:
    with open('data/fantasypros/projections/{}.json'.format(position), 'r') as f:
        projections_json = json.load(f)
    
    projections_df = pd.DataFrame(projections_json['players'])
    projections_dfs.append(projections_df)
    
    with open('data/fantasypros/consensus/{}.json'.format(position), 'r') as f:
        consensus_json = json.load(f)
    
    consensus_df = pd.DataFrame(consensus_json['players'])
    consensus_dfs.append(consensus_df)

In [6]:
projections = pd.concat(projections_dfs)
consensus = pd.concat(consensus_dfs)

In [7]:
# transform reddit data
reddit_submissions = reddit_submissions.explode('players').drop_duplicates()
reddit_submissions['body'] = reddit_submissions.selftext + ' ' + reddit_submissions.title
reddit_submissions = reddit_submissions[['body', 'players', 'created']].groupby('players').agg({'body': lambda x: ' '.join(x), 'created': 'count'})
reddit_submissions = reddit_submissions.rename(columns = {'created': 'submission_count', 'body': 'submission_text'})

reddit_comments = reddit_comments.explode('players').drop_duplicates()
reddit_comments = reddit_comments[['body', 'players', 'created']].groupby('players').agg({'body': lambda x: ' '.join(x), 'created': 'count'})
reddit_comments = reddit_comments.rename(columns = {'created': 'comment_count', 'body': 'comments_text'})

In [8]:
# transform twitter data
tweets = tweets.drop_duplicates()
tweets = tweets[['created', 'body', 'p_id']].groupby('p_id').agg({'body': lambda x: ' '.join(x), 'created': 'count'})
tweets = tweets.rename(columns = {'created': 'tweet_count', 'body': 'tweets_text'})

In [9]:
# explode json column "stats" into separate columns
projections = pd.concat([projections, projections.stats.apply(pd.Series)], axis = 1)
projections = pd.concat([projections, projections.stats.apply(pd.Series)], axis = 1)
projections = projections.drop(columns = ['mflid', 'name', 'position_id', 'team_id', 'filename', 'stats',
                                          'position_id', 'team_id', 'filename'])

In [10]:
consensus = consensus.drop(columns = ['player_name','sportsdata_id','player_team_id','player_position_id',
                                      'player_positions','player_short_name','player_eligibility','player_yahoo_positions',
                                      'player_page_url','player_filename','player_square_image_url','player_image_url',
                                      'player_yahoo_id','cbs_player_id','player_bye_week','rank_ecr'])

In [11]:
players

,player_id,player_name,short_name,first_name,last_name,reverse_name,position_id,positions,team_id,filename,...,image_url,rank_ecr,rank_adp,rank_ecr_ppr,rank_adp_ppr,rank_ecr_half,birthdate,birthdatetime,age,rookie
0,8000,Arizona Cardinals,Arizona,Arizona,Cardinals,"Cardinals, Arizona",DST,[DST],ARI,https://www.fantasypros.com/nfl/teams/arizona-...,...,https://images.fantasypros.com/images/nfl/team...,283,282,283,237,274,,,NaN,NaN
1,8010,Atlanta Falcons,Atlanta,Atlanta,Falcons,"Falcons, Atlanta",DST,[DST],ATL,https://www.fantasypros.com/nfl/teams/atlanta-...,...,https://images.fantasypros.com/images/nfl/team...,315,318,319,346,305,,,NaN,NaN
2,8020,Baltimore Ravens,Baltimore,Baltimore,Ravens,"Ravens, Baltimore",DST,[DST],BAL,https://www.fantasypros.com/nfl/teams/baltimor...,...,https://images.fantasypros.com/images/nfl/team...,160,151,161,120,167,,,NaN,NaN
3,8030,Buffalo Bills,Buffalo,Buffalo,Bills,"Bills, Buffalo",DST,[DST],BUF,https://www.fantasypros.com/nfl/teams/buffalo-...,...,https://images.fantasypros.com/images/nfl/team...,191,140,189,144,191,,,NaN,NaN
4,8040,Carolina Panthers,Carolina,Carolina,Panthers,"Panthers, Carolina",DST,[DST],CAR,https://www.fantasypros.com/nfl/teams/carolina...,...,https://images.fantasypros.com/images/nfl/team...,314,311,318,249,314,,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5711,23551,Graham Adomitis,G. Adomitis,Graham,Adomitis,"Adomitis, Graham",TE,[TE],IND,https://www.fantasypros.com/nfl/players/graham...,...,,0,0,0,0,0,1996-08-15,840067200,25.0,NaN
5712,23552,Michael Jacobson,M. Jacobson,Michael,Jacobson,"Jacobson, Michael",TE,[TE],SEA,https://www.fantasypros.com/nfl/players/michae...,...,,0,0,0,0,0,1997-04-04,860112000,24.0,NaN
5713,23553,KJ Sails,KJ Sails,KJ,Sails,"Sails, KJ",DB,[DB],LAC,https://www.fantasypros.com/nfl/players/kj-sai...,...,,0,0,0,0,0,1997-05-02,862531200,24.0,NaN
5714,23554,Dominique Martin,D. Martin,Dominique,Martin,"Martin, Dominique",CB,[CB],GB,https://www.fantasypros.com/nfl/players/domini...,...,,0,0,0,0,0,1993-11-25,754185600,27.0,NaN


In [12]:
players = players.set_index('player_id').join(consensus.set_index('player_id'))
players = players.join(projections.set_index('fpid'))
players = players.join(tweets)
players = players.reset_index()
players = players.set_index('player_name').join(reddit_submissions)
players = players.join(reddit_comments).reset_index()

In [13]:
players

,index,player_id,short_name,first_name,last_name,reverse_name,position_id,positions,team_id,filename,...,def_safety,def_ff,def_fr,def_retd,tweets_text,tweet_count,submission_text,submission_count,comments_text,comment_count
0,A'Shawn Robinson,15503,A. Robinson,A'Shawn,Robinson,"Robinson, A'Shawn",DT,[DT],LAR,https://www.fantasypros.com/nfl/players/a-shaw...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Bouye,11983,A.J. Bouye,A.J.,Bouye,"Bouye, A.J.",CB,[CB],CAR,https://www.fantasypros.com/nfl/players/aj-bou...,...,NaN,NaN,NaN,NaN,NaN,NaN,[**Denver Broncos**](https://www.reddit.com/r/...,1.0,NaN,NaN
2,A.J. Brown,18218,A.J. Brown,A.J.,Brown,"Brown, A.J.",WR,[WR],TEN,https://www.fantasypros.com/nfl/players/aj-bro...,...,NaN,NaN,NaN,NaN,NaN,NaN,&amp;#x200B;\n\nhttps://preview.redd.it/s7vl5n...,2.0,**Matthew Stafford** \n\n\n&amp;#x200B;\n\n ...,1.0
3,A.J. Epenesa,19204,A.J. Epenesa,A.J.,Epenesa,"Epenesa, A.J.",DE,[DE],BUF,https://www.fantasypros.com/nfl/players/aj-epe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Green,9857,A.J. Green,A.J.,Green,"Green, A.J.",WR,[WR],ARI,https://www.fantasypros.com/nfl/players/aj-gre...,...,NaN,NaN,NaN,NaN,NaN,NaN,Feel free to drop your suggestions or your own...,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5711,Zeron Flemister,20605,Z. Flemister,Flemister,Zeron,"Zeron, Flemister",TE,[TE],FA,https://www.fantasypros.com/nfl/players/zeron-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5712,Zimari Manning,19902,Z. Manning,Zimari,Manning,"Manning, Zimari",WR,[WR],FA,https://www.fantasypros.com/nfl/players/zimari...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5713,Zonovan Knight,23038,Z. Knight,Zonovan,Knight,"Knight, Zonovan",RB,[RB],FA,https://www.fantasypros.com/nfl/players/zonova...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5714,Zuriel Smith,21619,Z. Smith,Smith,Zuriel,"Zuriel, Smith",WR,[WR],FA,https://www.fantasypros.com/nfl/players/zuriel...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
